### Preencher:
email.send_keys("") com seu email

senha.send_keys("") com sua senha 

db_url="" com o url da sua conexão

openai.api_key("")com sua chave do ChatGPT



In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from requests.exceptions import Timeout
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import openai
import psycopg2
import re
import time
import json
import requests


def find_Esperiencia():
    try:
        elemento=driver.find_elements(By.XPATH,"//span[@aria-hidden='true'][normalize-space()='Experiência']//parent::h2//parent::div//parent::div//parent::div//parent::div//following-sibling::div")[-1]
        return (elemento.text)
    except NoSuchElementException:
        return "Experiência não encontrada"
    except IndexError:
        return "Experiência não encontrada"

def find_Titulo_Do_Candidato():
    try:

        elemento=driver.find_element(By.XPATH,"//div[@class='text-body-medium break-words']")
        return (elemento.text)
    except NoSuchElementException:
        return "Titulo não encontrado"
    except IndexError:
        return "Titulo não encontrada"

def find_Organizacao_Atual():
    try:

        elemento=driver.find_element(By.XPATH,"//ul[@class='pv-text-details__right-panel']")
        return (elemento.text)
    except NoSuchElementException:
        return "Organização não encontrada"
    except IndexError:
        return "Organização não encontrada"

def find_Formacao_Academica():
    try:
        elemento=driver.find_elements(By.XPATH,"//span[@aria-hidden='true'][normalize-space()='Formação acadêmica']//parent::h2//parent::div//parent::div//parent::div//parent::div//following-sibling::div")[-1]
        return (elemento.text)
    except NoSuchElementException:
        return "Formação academica não encontrada"
    except IndexError:
        return "Formação não encontrada"

def find_Nome():
    elemento=driver.find_element(By.XPATH,"//h1[@class='text-heading-xlarge inline t-24 v-align-middle break-words']")
    return (elemento.text)
   

def find_Sobre():
    try:
        #time.sleep(2)
        elemento=driver.find_elements(By.XPATH,"//span[@aria-hidden='true'][normalize-space()='Sobre']//parent::h2//parent::div//parent::div//parent::div//parent::div//following-sibling::div")[-2]
        return (elemento.text)
    except NoSuchElementException:
        return "Sobre não encontrada"
    except IndexError:
        return "Sobre não encontrada"
    
def wait(url, xpath, max_attempts=3):
    for _ in range(max_attempts):
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
            return  # Elemento encontrado, saia da função
        except TimeoutException:
            driver.get(url)
            return
        
    


In [ ]:
# options = webdriver.EdgeOptions()
# options.add_argument('headless')
# # options.add_argument('disable-gpu')
driver=webdriver.Edge()
driver.get("https://www.linkedin.com/?trk=seo-authwall-base_nav-header-logo&original_referer=https%3A%2F%2Fwww.linkedin.com%2Fauthwall%3Ftrk%3Dqf%26original_referer%3D%26sessionRedirect%3Dhttps%253A%252F%252Fwww.linkedin.com%252F")
time.sleep(3)

assert "LinkedIn" in driver.title

# html_source = driver.page_source
## Salve o código-fonte HTML em um arquivo
# with open('pagina.html', 'w', encoding='utf-8') as file:
#     file.write(html_source)
# body = driver.find_element(By.TAG_NAME, 'body')
# body.send_keys(Keys.END)
# driver.save_screenshot('captura_de_tela.png')

email = driver.find_element(By.NAME, "session_key")
senha = driver.find_element(By.NAME, "session_password")
botao = driver.find_element(By.XPATH,"//button[@type='submit']")


email.send_keys("")
senha.send_keys("")
botao.click()
pessoas=[]

#//h1[@class='t-20 t-black t-normal mb2']

In [ ]:
pessoas=[]

In [ ]:
tempos=[]
start_time = time.time()

pagina_Pessoas="https://www.linkedin.com/search/results/people/?origin=CLUSTER_EXPANSION&page=1&sid=3Em"
driver.get(pagina_Pessoas)
xpath="//div[1]//div[1]//main[1]//div[1]//div[1]//div[1]//h2[1]//div[1]"

wait(pagina_Pessoas,xpath)


for npage in range(6):
    url_anterior=driver.current_url
    for ntable in range(3):
        try:
            lista_Pessoas=driver.find_element(By.XPATH,"//li["+str(ntable+1)+"]//div[1]//div[1]//div[2]//div[1]//div[1]//div[1]//span[1]//span[1]//a[1]//span[1]//span[1]")
            lista_Pessoas.click()
            
            wait(driver.current_url,"//span[@aria-hidden='true'][normalize-space()='Formação acadêmica']//parent::h2//parent::div//parent::div//parent::div//parent::div//following-sibling::div")
            pessoa={
                "Nome":find_Nome(),
                "Titulo":find_Titulo_Do_Candidato(),
                "Sobre":find_Sobre(),
                "Esperiencia":find_Esperiencia(),
                "Organização Atual":find_Organizacao_Atual(),
                "Formação acadêmica":find_Formacao_Academica(),
                "Link_Linkedin": driver.current_url
                }
            pessoas.append(pessoa)
            driver.get(url_anterior)
            wait(url_anterior,xpath)

        except NoSuchElementException:
            continue
    pagina_Pessoas='https://www.linkedin.com/search/results/people/?origin=CLUSTER_EXPANSION&page='+str(npage+2)+'&sid=3Em'       
    driver.get(pagina_Pessoas)
    wait(pagina_Pessoas,xpath)

with open("pessoas2.json", "a", encoding='utf-8') as file:
    json.dump(pessoas, file, indent=4, ensure_ascii=False)
file.close()

tempos.append(time.time()-start_time)
print(tempos)

In [ ]:
len(pessoas)

In [ ]:
with open('pessoas2.json', 'r', encoding='utf-8') as arquivo:
    seu_texto=arquivo.read()
blocos = re.split(r"\n{'Nome':", seu_texto)[1:]
pessoasarquivo=[]
lista_de_dicionarios = []

for bloco in blocos:
    bloco_com_nome = "{'Nome':" + bloco
    pessoa_dict = eval(bloco_com_nome)
    pessoasarquivo.append(pessoa_dict)

for pessoa in pessoasarquivo:
    print(pessoa)
print(len(pessoasarquivo))

In [ ]:
with open('pessoas2.json', 'r', encoding='utf-8') as arquivo:
    pessoas=json.load(arquivo)

In [ ]:
pessoas[0]["Nome"]

In [ ]:
print(pessoa["Nome"])
print("#################################")
print(pessoa["Titulo"])
print("#################################")
print(pessoa["Sobre"])
print("#################################")
print(pessoa["Esperiencia"])
print("#################################")
print(pessoa["Organização Atual"])
print("#################################")
print(pessoa["Formação acadêmica"])
print("#################################")
print(pessoa ['Link_Linkedin'])



In [ ]:

openai.api_key = ""

cha_results = []

for n, pessoa in enumerate(pessoas):
    retry_count = 0
    max_retries = 3  
    
    while retry_count < max_retries:
        try:
            resposta = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "user",
                        "content": f"""
                            Analise o CHA (Competencias , Habilidades , e Atitudes) dessa pessoa:
                            {pessoa}
                            
                            Responda dentro de um Json nesse modelo :
                            {{
                                "Competencias":[],
                                "Habilidades":[],
                                "Atitudes":[]
                            }}
                            
                            Tente descobrir todas as Competencias, Habilidades e Atitudes e não deixe nenhuma das chaves vazias.
                            """
                    }
                ]
            )
            
            mensagem_resposta = resposta['choices'][0]['message']['content']
            pessoa['CHA'] = mensagem_resposta
            
            cha_results.append({"Nome": pessoa["Nome"], "CHA": mensagem_resposta})
            
            with open("cha_results2.json", "a") as file:
                json.dump(cha_results, file, indent=4)
    
            print(f"CHA analysis saved for pessoa {n + 1}.")
            
            
            break
        
        except requests.exceptions.Timeout:
            retry_count += 1
            print(f"Timeout occurred while processing pessoa {n + 1}. Retrying (Attempt {retry_count}).")

    if retry_count == max_retries:
        print(f"Max retries reached for pessoa {n + 1}. Skipping.")

print("All CHA analyses saved to cha_results.json.")


In [ ]:
pessoa

In [ ]:
with open('pessoasComCha', "w") as arquivo:
    json.dump(pessoas, arquivo)
    arquivo.close()

In [ ]:

# Database connection parameters
db_url = ""

try:
    conn = psycopg2.connect(db_url)
    cursor = conn.cursor()
except psycopg2.Error as e:
    print("Error connecting to the database:", e)

try:
    # Insert the sample profile data into the table
    for pessoa in pessoas:
        cursor.execute("""
            INSERT INTO profile_linkedin (name, title, about, experience, current_organization, academic_background, linkedin_link, CHA)
            VALUES (%(Nome)s, %(Titulo)s, %(Sobre)s, %(Esperiencia)s, %(Organização Atual)s, %(Formação acadêmica)s, %(Link_Linkedin)s, %(CHA)s);
        """, pessoa)
    
    # Commit the transaction
    conn.commit()

    print("Profile data inserted successfully!")

except psycopg2.Error as e:
    print("Error inserting data:", e)

finally:
    cursor.close()
    conn.close()





In [ ]:
driver=webdriver.Edge()
driver.get("https://auth0.openai.com/u/login/identifier?state=hKFo2SBsYWxOamNCYzRFTlNnXzA4ckV2ZldMSVFrVVJ6OWFFQqFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIGlCUWxQU1JwaXluLVcyM1pOMzRIRTl1WWthZ18zZTRoo2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q")
email=driver.find_element(By.XPATH,"//input[@name='username']")
email.send_keys("lucca14santiago@gmail.com")
botao=driver.find_element(By.XPATH,"//button[normalize-space()='Continue']")
botao.click()
senha=driver.find_element(By.XPATH,"//input[@name='password']")
senha.send_keys("147852369")
botao=driver.find_element(By.XPATH,"//button[@class='c4c7ab148 cedb58860 c9857f26f c00f6c27d _button-login-password']")
botao.click()




